In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_rows', None)


# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las tablas relevantes (tres columnas: Habilidad lectora, Matemáticas, Ciencias)
table = soup.find_all('table', {'border': '1'})[0]  # Primera tabla con borde

# Extraer datos de la tabla
data = []
columns = ["Posición", "Comunidad Autónoma", "Habilidad lectora", "Matemáticas", "Ciencias"]

# Extraer las subtablas dentro de la principal
subtables = table.find_all('table', {'border': '0'})

for i, subtable in enumerate(subtables):
    rows = subtable.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 3:  # Comprobar si la fila tiene datos válidos
            posicion = cols[0].get_text(strip=True).replace('.', '')  # Quitar el punto
            comunidad = cols[1].find('a').get_text(strip=True) if cols[1].find('a') else cols[1].get_text(strip=True)
            puntuacion = cols[2].get_text(strip=True)
            data.append([posicion, comunidad, puntuacion])

# Crear un DataFrame
dataframe = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación"])

# Filtrar filas válidas (solo números en la columna "Posición")
dataframe = dataframe[dataframe['Posición'].str.isdigit()]

# Convertir "Posición" a entero y "Puntuación" a flotante
dataframe['Posición'] = dataframe['Posición'].astype(int)
dataframe['Puntuación'] = dataframe['Puntuación'].astype(float)

# Agregar la columna de habilidades
dataframe['Habilidad'] = ['Habilidad lectora'] * 19 + ['Matemáticas'] * 19 + ['Ciencias'] * 19

dataframe





,Posición,Comunidad Autónoma,Puntuación,Habilidad
0,1,,503.0,Habilidad lectora
1,2,,503.0,Habilidad lectora
2,3,,498.0,Habilidad lectora
3,4,,498.0,Habilidad lectora
4,5,,497.0,Habilidad lectora
5,6,,495.0,Habilidad lectora
6,7,,494.0,Habilidad lectora
8,8,,490.0,Habilidad lectora
9,9,,488.0,Habilidad lectora
10,10,,486.0,Habilidad lectora


In [ ]:
# Pivotar la tabla para que cada habilidad sea una columna
result_df = dataframe.pivot(index='Comunidad Autónoma', columns='Habilidad', values='Puntuación').reset_index()

# Mostrar el DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Resultados Informe PISA 2022", dataframe=result_df)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar la tabla principal (border="1")
table = soup.find_all('table', {'border': '1'})[0]  # Primera tabla con borde

# Hallar las 3 subtablas internas (una por cada habilidad)
subtables = table.find_all('table', {'border': '0'})

# Variable para ir acumulando todos los registros
data = []

# Mapeo de índice de subtabla a nombre de la habilidad
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

# Recorremos cada una de las subtablas
for i, subtable in enumerate(subtables):
    # Determinamos la habilidad en función del índice
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    rows = subtable.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        # Las filas relevantes tienen al menos 3 celdas: posición, comunidad, puntuación
        if len(cols) >= 3:
            posicion   = cols[0].get_text(strip=True).replace('.', '')
            # Intentamos extraer la comunidad autónoma desde un enlace <a> si existe
            comunidad  = cols[1].find('a').get_text(strip=True) if cols[1].find('a') else cols[1].get_text(strip=True)
            puntuacion = cols[2].get_text(strip=True)
            
            # Guardamos la información, incluyendo la habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# Convertimos a DataFrame
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# --- Si quieres mantener la información tal cual en un formato "largo" (long format), basta con esto:
print("=== Formato largo (todas las filas, incluidas medias) ===")
print(df)
print()

# --- Si te interesa pivotar la información para ver las tres habilidades en columnas,
#     podemos usar pivot_table. Esto unificará filas con el mismo 'Comunidad Autónoma'
#     en una sola. Se usan funciones de agregación en caso de duplicados.

# Antes, convertimos "Puntuación" a float (cuando se pueda) para que no dé error:
def to_float(x):
    try:
        return float(x)
    except ValueError:
        return None

df["Puntuación"] = df["Puntuación"].apply(to_float)

df


=== Formato largo (todas las filas, incluidas medias) ===
   Posición Comunidad Autónoma Puntuación          Habilidad
0         1                           503  Habilidad lectora
1         2                           503  Habilidad lectora
2         3                           498  Habilidad lectora
3         4                           498  Habilidad lectora
4         5                           497  Habilidad lectora
..      ...                ...        ...                ...
58       15                           469           Ciencias
59       16                           461           Ciencias
60       17                           452           Ciencias
61       18                           426           Ciencias
62       19                           404           Ciencias

[63 rows x 4 columns]



,Posición,Comunidad Autónoma,Puntuación,Habilidad
0,1,,503.0,Habilidad lectora
1,2,,503.0,Habilidad lectora
2,3,,498.0,Habilidad lectora
3,4,,498.0,Habilidad lectora
4,5,,497.0,Habilidad lectora
...,...,...,...,...
58,15,,469.0,Ciencias
59,16,,461.0,Ciencias
60,17,,452.0,Ciencias
61,18,,426.0,Ciencias


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar la tabla principal (border="1")
table = soup.find_all('table', {'border': '1'})[0]  # Primera tabla con borde

# Hallar las 3 subtablas internas (una por cada habilidad)
subtables = table.find_all('table', {'border': '0'})

# Variable para ir acumulando todos los registros
data = []

# Mapeo de índice de subtabla a nombre de la habilidad
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

# Recorremos cada una de las subtablas
for i, subtable in enumerate(subtables):
    # Determinamos la habilidad en función del índice
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    rows = subtable.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        # Las filas relevantes tienen al menos 3 celdas: posición, comunidad, puntuación
        if len(cols) >= 3:
            posicion   = cols[0].get_text(strip=True).replace('.', '')
            # Intentamos extraer la comunidad autónoma desde un enlace <a> si existe
            comunidad  = cols[1].find('a').get_text(strip=True) if cols[1].find('a') else cols[1].get_text(strip=True)
            puntuacion = cols[2].get_text(strip=True)
            
            # Guardamos la información, incluyendo la habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# Convertimos a DataFrame
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# --- Si quieres mantener la información tal cual en un formato "largo" (long format), basta con esto:
print("=== Formato largo (todas las filas, incluidas medias) ===")
print(df)
print()

# --- Si te interesa pivotar la información para ver las tres habilidades en columnas,
#     podemos usar pivot_table. Esto unificará filas con el mismo 'Comunidad Autónoma'
#     en una sola. Se usan funciones de agregación en caso de duplicados.

# Antes, convertimos "Puntuación" a float (cuando se pueda) para que no dé error:
def to_float(x):
    try:
        return float(x)
    except ValueError:
        return None

df["Puntuación"] = df["Puntuación"].apply(to_float)

# Hacemos el pivot. Usamos `aggfunc='first'` para evitar error si hay filas duplicadas.
result_df = df.pivot_table(
    index='Comunidad Autónoma',
    columns='Habilidad',
    values='Puntuación',
    aggfunc='first'
).reset_index()

print("=== Formato pivotado (tabular), incluyendo promedios y totales ===")
print(result_df)

# Si quieres ver el DataFrame en un Jupyter Notebook o IPython:
# display(result_df)

# Si deseas exportarlo a CSV:
# result_df.to_csv("resultados_pisa_2022_todo.csv", index=False)


=== Formato largo (todas las filas, incluidas medias) ===
   Posición Comunidad Autónoma Puntuación          Habilidad
0         1                           503  Habilidad lectora
1         2                           503  Habilidad lectora
2         3                           498  Habilidad lectora
3         4                           498  Habilidad lectora
4         5                           497  Habilidad lectora
..      ...                ...        ...                ...
58       15                           469           Ciencias
59       16                           461           Ciencias
60       17                           452           Ciencias
61       18                           426           Ciencias
62       19                           404           Ciencias

[63 rows x 4 columns]

=== Formato pivotado (tabular), incluyendo promedios y totales ===
Habilidad Comunidad Autónoma  Ciencias  Habilidad lectora  Matemáticas
0                                516.0          

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Localizar la tabla con border="1" que contiene todas las subtablas
table = soup.find_all('table', {'border': '1'})[0]

# Identificar las subtablas (una por cada habilidad)
subtables = table.find_all('table', {'border': '0'})

# Diccionario para nombrar las habilidades en función de la subtabla
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

data = []

# Recorrer cada subtabla (cada habilidad)
for i, subtable in enumerate(subtables):
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    # Para cada fila <tr>...
    rows = subtable.find_all('tr')
    for row in rows:
        # ...buscamos todas las celdas, sean <td> o <th>
        cells = row.find_all(['td', 'th'])
        
        # Nos aseguramos de que haya al menos 3 celdas
        if len(cells) >= 3:
            # 1) Posición (col 0)
            posicion = cells[0].get_text(strip=True).replace('.', '')
            
            # 2) Comunidad (col 1)
            a_tag = cells[1].find('a')
            if a_tag:
                comunidad = a_tag.get_text(strip=True)
            else:
                comunidad = cells[1].get_text(strip=True)
            
            # 3) Puntuación (col 2)
            puntuacion = cells[2].get_text(strip=True)
            
            # Almacenamos en 'data' junto con la habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# Convertimos 'data' a un DataFrame
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# --- FORMATO LARGO ---
print("=== Formato largo (todas las filas) ===")
print(df)
print()

# Si deseamos trabajar con valores numéricos en 'Puntuación', hacemos:
def to_float(x):
    try:
        return float(x)
    except ValueError:
        return None

df['Puntuación'] = df['Puntuación'].apply(to_float)

# --- FORMATO PIVOTADO ---
# Agrupamos por 'Comunidad Autónoma' y mostramos cada habilidad como columna
result_df = df.pivot_table(
    index='Comunidad Autónoma',
    columns='Habilidad',
    values='Puntuación',
    aggfunc='first'  # si hubiese duplicados, tomamos el primero
).reset_index()

print("=== Formato pivotado (tabular), incluyendo promedios y totales ===")
print(result_df)

# Para guardarlo en CSV:
# result_df.to_csv("resultados_pisa_2022_completo.csv", index=False)


=== Formato largo (todas las filas) ===
   Posición Comunidad Autónoma Puntuación          Habilidad
0         1                           503  Habilidad lectora
1         2                           503  Habilidad lectora
2         3                           498  Habilidad lectora
3         4                           498  Habilidad lectora
4         5                           497  Habilidad lectora
..      ...                ...        ...                ...
58       15                           469           Ciencias
59       16                           461           Ciencias
60       17                           452           Ciencias
61       18                           426           Ciencias
62       19                           404           Ciencias

[63 rows x 4 columns]

=== Formato pivotado (tabular), incluyendo promedios y totales ===
Habilidad Comunidad Autónoma  Ciencias  Habilidad lectora  Matemáticas
0                                516.0              503.0        5

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Tomamos la tabla principal con border="1"
table = soup.find_all('table', {'border': '1'})[0]

# Las 3 subtablas (una por cada habilidad)
subtables = table.find_all('table', {'border': '0'})

# Diccionario para asignar nombre de habilidad por índice
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

data = []

# Recorremos cada subtabla (cada habilidad)
for i, subtable in enumerate(subtables):
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    # Para cada fila de la subtabla
    rows = subtable.find_all('tr')
    for row in rows:
        # Buscamos todas las celdas (td y th), por si acaso
        cells = row.find_all(['td','th'])
        # Necesitamos al menos 3 celdas: posición, comunidad, puntuación
        if len(cells) >= 3:
            # 1) Posición
            posicion = cells[0].get_text(strip=True).replace('.', '')

            # 2) Comunidad Autónoma
            #    - Obtenemos todos los <a> de la celda
            #    - Si hay varios, el último suele ser el que tiene el nombre real de la comunidad
            all_links = cells[1].find_all('a', href=True)
            if all_links:
                comunidad = all_links[-1].get_text(strip=True)
            else:
                # Si no hay enlace, puede ser un texto como "Media España", "Media OCDE", etc.
                comunidad = cells[1].get_text(strip=True)

            # 3) Puntuación
            puntuacion = cells[2].get_text(strip=True)

            data.append([posicion, comunidad, puntuacion, habilidad])

# Convertimos 'data' a DataFrame
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# (Opcional) Mostrar en formato largo
print("=== FORMATO LARGO (todas las filas, sin filtrar) ===")
print(df)
print()

# Convertir la columna 'Puntuación' a float si queremos trabajar con números
def to_float(x):
    try:
        return float(x)
    except ValueError:
        return None

df['Puntuación'] = df['Puntuación'].apply(to_float)

# Poner en formato 'ancho' (pivotar), para ver cada habilidad en una columna
result_df = df.pivot_table(
    index='Comunidad Autónoma',
    columns='Habilidad',
    values='Puntuación',
    aggfunc='first'
).reset_index()

print("=== FORMATO PIVOTADO (incluye promedios y medias) ===")
print(result_df)

# Exportar a CSV si se desea
# result_df.to_csv("resultados_pisa_2022_con_todos_los_textos.csv", index=False)


=== FORMATO LARGO (todas las filas, sin filtrar) ===
   Posición    Comunidad Autónoma Puntuación          Habilidad
0         1       Castilla y León        503  Habilidad lectora
1         2   Comunidad de Madrid        503  Habilidad lectora
2         3              Cataluña        498  Habilidad lectora
3         4              La Rioja        498  Habilidad lectora
4         5               Navarra        497  Habilidad lectora
5         6                Aragón        495  Habilidad lectora
6         7            País Vasco        494  Habilidad lectora
7                      Media OCDE        493  Habilidad lectora
8         8              Asturias        490  Habilidad lectora
9         9             Cantabria        488  Habilidad lectora
10       10               Galicia        486  Habilidad lectora
11                   Media España        481  Habilidad lectora
12       11      Región de Murcia        480  Habilidad lectora
13       12  Comunidad Valenciana        477  Habil

In [10]:
df.head(63)

,Posición,Comunidad Autónoma,Puntuación,Habilidad
0,1,Castilla y León,503.0,Habilidad lectora
1,2,Comunidad de Madrid,503.0,Habilidad lectora
2,3,Cataluña,498.0,Habilidad lectora
3,4,La Rioja,498.0,Habilidad lectora
4,5,Navarra,497.0,Habilidad lectora
5,6,Aragón,495.0,Habilidad lectora
6,7,País Vasco,494.0,Habilidad lectora
7,,Media OCDE,493.0,Habilidad lectora
8,8,Asturias,490.0,Habilidad lectora
9,9,Cantabria,488.0,Habilidad lectora


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Se localiza la tabla principal con border="1"
main_table = soup.find_all('table', {'border': '1'})[0]

# Cada habilidad está en una subtabla (border="0")
subtables = main_table.find_all('table', {'border': '0'})

# Asignamos nombre de habilidad según el índice
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

data = []

# Recorremos cada subtabla y sus filas
for i, subtable in enumerate(subtables):
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    rows = subtable.find_all('tr')
    
    for row in rows:
        # Obtenemos todas las celdas (tanto <td> como <th>)
        cells = row.find_all(['td','th'])
        
        # Necesitamos al menos 3 celdas: posición, comunidad, puntuación
        if len(cells) >= 3:
            # 1) Posición (col 0)
            #    No se hace limpieza: se toma tal cual (sin convertir a número)
            posicion = cells[0].get_text(strip=True)

            # 2) Comunidad Autónoma (col 1)
            #    A menudo hay dos <a>: uno para la bandera y otro para el nombre.
            #    Para capturar el nombre de la comunidad, nos quedamos con el último <a>.
            all_links = cells[1].find_all('a', href=True)
            if all_links:
                comunidad = all_links[-1].get_text(strip=True)
            else:
                comunidad = cells[1].get_text(strip=True)

            # 3) Puntuación (col 2)
            puntuacion = cells[2].get_text(strip=True)

            # Guardamos en la lista con la habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# Creamos el DataFrame sin conversión ni limpieza adicional
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# Mostramos el DataFrame
df




,Posición,Comunidad Autónoma,Puntuación,Habilidad
0,1.,Castilla y León,503,Habilidad lectora
1,2.,Comunidad de Madrid,503,Habilidad lectora
2,3.,Cataluña,498,Habilidad lectora
3,4.,La Rioja,498,Habilidad lectora
4,5.,Navarra,497,Habilidad lectora
5,6.,Aragón,495,Habilidad lectora
6,7.,País Vasco,494,Habilidad lectora
7,,Media OCDE,493,Habilidad lectora
8,8.,Asturias,490,Habilidad lectora
9,9.,Cantabria,488,Habilidad lectora


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. URL de la página
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# 2. Realizar la solicitud
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 3. Localizar el encabezado h3 con id="2022"
h3_2022 = soup.find("h3", {"id": "2022"})

# 4. Encontrar la tabla con border="1" que aparece justo después del h3 de 2022
table_2022 = h3_2022.find_next("table", {"border": "1"})

# 5. Dentro de esa tabla, hallar sus subtablas con border="0" (3 subtablas para las 3 habilidades)
subtables = table_2022.find_all("table", {"border": "0"})

# Para etiquetar cada subtabla con el nombre de la habilidad
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

# Aquí guardamos todas las filas (sin limpiar ni transformar)
data = []

# 6. Recorrer las subtablas y sus filas
for i, subtable in enumerate(subtables):
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    rows = subtable.find_all("tr")
    for row in rows:
        # Obtenemos todas las celdas
        cells = row.find_all('td')
        # La estructura esperada: 3 celdas (Posición / Comunidad / Puntuación)
        if len(cells) >= 3:
            # a) Posición (sin limpiar ni convertir a número)
            posicion = cells[0].get_text(strip=True)
            
            # b) Comunidad Autónoma
            #    Algunas celdas tienen 2 enlaces (bandera y nombre). Tomamos el último <a>
            all_links = cells[1].find_all("a", href=True)
            if all_links:
                comunidad = all_links[-1].get_text(strip=True)
            else:
                comunidad = cells[1].get_text(strip=True)
            
            # c) Puntuación (sin convertir a float)
            puntuacion = cells[2].get_text(strip=True)
            
            # Añadimos la fila con su habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# 7. Crear el DataFrame final con los datos en bruto
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# 8. Mostrar el DataFrame (sin limpieza ni transformación)
print(df)

# Si deseas guardarlo a CSV sin transformar:
# df.to_csv("resultados_pisa_2022_sin_limpieza.csv", index=False)


   Posición    Comunidad Autónoma Puntuación          Habilidad
0        1.       Castilla y León        498  Habilidad lectora
1        2.              Asturias        497  Habilidad lectora
2        3.   Comunidad de Madrid        496  Habilidad lectora
3        4.             Cantabria        494  Habilidad lectora
4        5.                Aragón        488  Habilidad lectora
5        6.              La Rioja        487  Habilidad lectora
6        7.               Galicia        485  Habilidad lectora
7        8.  Comunidad Valenciana        482  Habilidad lectora
8        9.               Navarra        478  Habilidad lectora
9                      Media OCDE        476  Habilidad lectora
10                    Promedio EU        475  Habilidad lectora
11                   Media España        474  Habilidad lectora
12      10.        Islas Baleares        472  Habilidad lectora
13      11.      Región de Murcia        468  Habilidad lectora
14      12.           Extremadura       